In [1]:
import scipy
import numpy as np
from sklearn.neighbors import KernelDensity
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.cluster import estimate_bandwidth
from sklearn.cluster import MeanShift, estimate_bandwidth

import pandas as pd

from scipy import stats
from scipy.stats import beta
from math import sin
from random import randint
from IPython.display import clear_output
import matplotlib.pyplot as plt
import itertools as it

import plotly
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
init_notebook_mode(connected=True)

import collections

def recursively_default_dict():
        return collections.defaultdict(recursively_default_dict)

from matplotlib.collections import BrokenBarHCollection
import re

from structure_tools.Modules_tools import return_fsts

PCA_color_ref= ['darkseagreen','crimson', 'darkorange', 'darkblue', 'darkcyan',
            'darkgoldenrod', 'darkgray', 'darkgrey', 'darkgreen',
            'darkkhaki', 'darkmagenta', 'darkolivegreen', 'darkorange',
            'darkorchid', 'darkred', 'darksalmon', 'darkseagreen',
            'darkslateblue', 'darkslategray', 'darkslategrey',
            'darkturquoise', 'darkviolet', 'deeppink']

## PCA inverse transformation.

Principal component analysis (PCA) is an extremely useful tool in population genetics. Indeed in any study to require
a summary of sample covariance. In population genetics the variance between populations is informative of the evolutionary 
history captured in a given data sets. In conditions of pure diffusion a direct relationship can even be found between divergence, as measured in coalescent time or Fst. 

The possibility of working in an euclidean space also opens the door for density based methods.

Another interesting side to PC transformation is the possibility of inverse transformation to obtain a raw data from feature space coordinates. 

However, it is likely that the accuracy of inverse transformation depends not only on the information retained during transformation but the information present in the data set itself.

Before we incorporate PCA inverse transformation in our analyses, its limits must be understood. 

In this notebook we will study the accuracy of inverse transformation in reproducing raw SNP data. 



### vcf Data

Jupyter notebook for the local analysis of genetic data stored in .vcf format.

Perform analysis of structure across data set, followed by a more detailed study of variation across local genomic windows.

#### Input

In [2]:
from structure_tools.vcf_geno_tools import simple_read_vcf

vcf_file= 'data_cleanRefs_simple_Admx.vcf'

genotype, summary, info_save= simple_read_vcf(vcf_file,row_info= 5,header_info= 9,phased= True)

print('Number of markers: {}'.format(genotype.shape[1]))
print('Number of individuals: {}'.format(genotype.shape[0]))

Number of markers: 40000
Number of individuals: 130


In [3]:
summary.head()

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT
0,1,8,1,A,T,.,PASS,.,GT:AD:DP
1,1,33,2,A,T,.,PASS,.,GT:AD:DP
2,1,74,3,A,T,.,PASS,.,GT:AD:DP
3,1,83,4,A,T,.,PASS,.,GT:AD:DP
4,1,87,5,A,T,.,PASS,.,GT:AD:DP


### Global variation

Perform PCA across data set.

Perform Mean shift clustering to attempt to extract genetically coherent groups of accessions.

These will later be used for supervised analysis.

In [4]:

## Perform PCA
n_comp= 3
pca = PCA(n_components=n_comp, whiten=False,svd_solver='randomized')

feats= pca.fit_transform(genotype)

In [5]:
## perform MeanShift clustering.
bandwidth = estimate_bandwidth(feats, quantile=0.1)

ms = MeanShift(bandwidth=bandwidth, bin_seeding=False, cluster_all=True, min_bin_freq=45)
ms.fit(feats)
labels1 = ms.labels_
label_select = {y:[x for x in range(len(labels1)) if labels1[x] == y] for y in sorted(list(set(labels1)))}
###

In [7]:
from structure_tools.Tutorial_subplots import plot_global_pca

plot_global_pca(feats,label_select,PCA_color_ref,title= 'global_pca',height= 500,width= 950)